In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('yearly_flow_data_by_countries.csv').iloc[:, 1:]
df.head(1)

,Economy_Label,Flow_Label,Year,SitcRev3Product_Label,Partner_Label,total_values
0,China,Exports,2016,"Travel goods, handbags & similar containers",Afghanistan,2235


In [6]:
df.SitcRev3Product_Label.unique()

array(['Travel goods, handbags & similar containers',
       'Monofilaments, of plastics, cross-section > 1mm',
       'Plates, sheets, films, foil & strip, of plastics',
       'Other plastics, in primary forms', 'Plastics in primary forms',
       'Waste, parings and scrap, of plastics',
       'Travel goods, handbags, etc.',
       'Tubes, pipes and hoses of plastics',
       'Articles, n.e.s., of plastics', 'Plastics in non-primary forms',
       'Cinematograph films, exposed & developed'], dtype=object)

In [132]:
def classify_country(x):
    if x == 'China':
        return 'China'
    elif x == 'India':
        return 'India'
    elif x == 'United States of America':
        return 'United States of America'
    else:
        return 'EU'
def classify_product(x):
    product_info = {'314910': 'textile bag',
                   '322220':'paper bag',
                   '326111':'plastic bag'}
    return product_info.get(x)

def convert_price(row):
    if row['Quantity Description'] == 'number':
        return row['price'] * 1000
    elif row['Quantity Description'] == 'thousand units':
        return row['price']
    elif row['Quantity Description'] == 'kilograms':
        return row['price'] * 127/2
    
def convert_qty(row):
    if row['Quantity Description'] == 'number':
        return row['qty'] / 1000
    elif row['Quantity Description'] == 'thousand units':
        return row['qty']
    elif row['Quantity Description'] == 'kilograms':
        return row['qty'] * 2/127

In [139]:

price = pd.read_csv('price.csv')
price['NAIC Number'] = price['NAIC Number'].astype(str)
price['product'] = price['NAIC Number'].apply(classify_product)

price = price[((price['NAIC Number']=='314910')&(price['Quantity Description']=='number'))|\
              ((price['NAIC Number']=='322220')&(price['Quantity Description']=='kilograms'))|\
             ((price['NAIC Number']=='326111')&(price['Quantity Description']=='thousand units'))].copy()
price = price.drop(columns = ['Data Type','NAIC Number'])
price = pd.melt(price, id_vars = ['Country','product','Quantity Description'], value_name='price', var_name = 'Year')
price['Year'] = price['Year'].apply(lambda x: x.split(' ')[1])
price['price'] = price.apply(convert_price, axis = 1)
price = price.rename(columns = {'price':'price_per_thousands'})
price

,Country,product,Quantity Description,Year,price_per_thousands
0,Austria,textile bag,number,2015,5740.000
1,Austria,paper bag,kilograms,2015,207.645
2,Austria,plastic bag,thousand units,2015,35.140
3,Belgium,textile bag,number,2015,7940.000
4,Belgium,paper bag,kilograms,2015,455.295
...,...,...,...,...,...
499,Spain,paper bag,kilograms,2020,149.860
500,Spain,plastic bag,thousand units,2020,9.020
501,Sweden,textile bag,number,2020,21610.000
502,Sweden,paper bag,kilograms,2020,234.950


In [140]:
qty = pd.read_csv('qty.csv')
qty = qty.dropna()
qty['NAIC Number'] = qty['NAIC Number'].astype(int)
qty['NAIC Number'] = qty['NAIC Number'].astype(str)
qty['product'] = qty['NAIC Number'].apply(classify_product)

qty = qty[((qty['NAIC Number']=='314910')&(qty['Quantity Description']=='number'))|\
              ((qty['NAIC Number']=='322220')&(qty['Quantity Description']=='kilograms'))|\
             ((qty['NAIC Number']=='326111')&(qty['Quantity Description']=='thousand units'))].copy()
qty = qty.drop(columns = ['Data Type','NAIC Number'])
qty = pd.melt(qty, id_vars = ['Country','product','Quantity Description'], value_name='qty', var_name = 'Year')
qty['Year'] = qty['Year'].apply(lambda x: x.split(' ')[1])
qty['qty'] = qty.apply(convert_qty, axis = 1)
qty = qty.rename(columns = {'qty':'qty_thousands'})
qty

,Country,product,Quantity Description,Year,qty_thousands
0,Austria,textile bag,number,2015,0.002420
1,Austria,paper bag,kilograms,2015,24.699685
2,Austria,plastic bag,thousand units,2015,3.080000
3,Belgium,textile bag,number,2015,0.000850
4,Belgium,paper bag,kilograms,2015,21.446772
...,...,...,...,...,...
499,Spain,paper bag,kilograms,2020,394.748661
500,Spain,plastic bag,thousand units,2020,594.230000
501,Sweden,textile bag,number,2020,0.001250
502,Sweden,paper bag,kilograms,2020,37.690551


In [145]:
merged = pd.merge(qty,price, on = ['Country', 'product', 'Quantity Description', 'Year'])
merged['country_category'] = merged['Country'].apply(classify_country)
merged.to_csv('trade_flow_data.csv')

In [84]:
price.groupby('NAIC Number')['Quantity Description'].unique()

NAIC Number
314910            [kilograms, number]
322220     [kilograms, square meters]
326111    [kilograms, thousand units]
Name: Quantity Description, dtype: object